In [ ]:
# 구글 드라이브 접근 코드
from google.colab import drive
drive.mount('/content/drive')
My_path = '/content/drive/MyDrive/LG_Academy/AI_Code/'

## 라이브러리 import

In [ ]:
import numpy       as np
import pandas      as pd
import matplotlib.pyplot as plt
import pywt

## 데이터 불러오기

In [ ]:
Data = pd.read_csv(My_path + 'ExampleData.csv', sep=',', header=None)
Data.shape

In [ ]:
# 예제데이터 센서데이터 (1열 ~ 2열)
plt.figure(figsize=(20,6))

plt.subplot(2,1,1) # x 방향 가속도 데이터
plt.plot(Data.iloc[:,0] , Data.iloc[:,1], color='r')
plt.ylabel('Acceleration (g)',fontsize=15, color='r')
plt.legend(['Acc_x'], loc = 'upper right')
plt.grid()

plt.subplot(2,1,2) # y 방향 가속도 데이터
plt.plot(Data.iloc[:,0] , Data.iloc[:,2], color='b')
plt.ylabel('Acceleration (g)',fontsize=15, color='b')
plt.legend(['Acc_y'], loc = 'upper right')
plt.grid()

plt.show()

## Wavelet Transform 수행

Wavelet Transfrom 실행 위한 파라미터 설정

In [ ]:
MotherWavelet = pywt.Wavelet('haar')   # Mother wavelet (모함수) 지정
Level   = 8                            # Wavelet 분해 레벨 지정

Wavelet Transform 실행 : Coefficient 생성

In [ ]:
Data_Target = Data.iloc[:,1] # 시계열 데이터 센서 종류 선택
Coefficient = pywt.wavedec(Data_Target, MotherWavelet, level=Level, axis=0)
Coefficient

In [ ]:
# 데이터 프레임 형태로 확인
Coefficient_df = pd.DataFrame(Coefficient)
Coefficient_df

각 WT Coefficent (주파수 영역별 신호) 별 데이터 사이즈 확인

In [ ]:
print('Size of a',Level, ' = ', len(Coefficient[0]))  # 첫번째 Coefficient : 최저주파수 신호 (a-최대레벨)

for i in range(1,Level+1):
    print('Size of d',Level+1-i , ' = ', len(Coefficient[i])) # 두번째 Coefficient(d-최대레벨) ~ 마지막 Coefficient(d1)

각 WT Coefficent (주파수 영역별 신호) 별 주파수 대역 확인

In [ ]:
Fs          = 20000
Fs_d1       = int(Fs/2)
Fs_range    = np.zeros([Level+1], dtype = int)
Fs_range[0] = Fs_d1

print('Frequency range of d 1', '=', Fs_range[0], '~', Fs)

for i in range(0, Level-1):
    Fs_range[i+1] = int(Fs_range[i]/2)

    print('Frequency range of d', i+2, '=', Fs_range[i+1], '~', Fs_range[i])

print('Frequency range of a', i+2, '=', Fs_range[i+2], '~', Fs_range[i+1])

### Wavelet Transfrom 결과 그래프 확인

In [ ]:
plt.figure(figsize=(15,20))

plt.subplot(Level+2,1,1)
plt.plot(Data.iloc[:,0], Data_Target, color='r')
plt.ylabel('Raw data')
plt.grid()

plt.subplot(Level+2,1,2)
Time_temp = np.arange( 0 , 3 + 3/(len(Coefficient[0])-1) , 3/(len(Coefficient[0])-1) )
plt.plot(Time_temp , Coefficient[0])
plt.ylabel('a %d' %(Level))
plt.grid()

plt.subplot(Level+2,1,3)
Time_temp = np.arange( 0 , 3 + 3/(len(Coefficient[0])-1) , 3/(len(Coefficient[0])-1) )
plt.plot(Time_temp , Coefficient[1])
plt.ylabel('d %d' %(Level))
plt.grid()

for k in range(2,Level+1):
    plt.subplot(Level+2,1,k+2)
    Time_temp = np.arange( 0 , 3 + 3/(len(Coefficient[k])-1) , 3/(len(Coefficient[k])-1) )
    plt.plot(Time_temp , Coefficient[k])
    plt.ylabel('d %d'%(Level+1-k))
    plt.grid()

plt.show()